In [2]:
import numpy as np
import scipy
from scipy.special import gamma
from scipy.special import beta
import scipy.integrate as integrate

In [300]:
def genParam(n, randomstate) :
    r = np.random.RandomState(seed=randomstate)
    
    alpha = r.exponential(1,(n,3))
    ksi = np.abs(r.standard_cauchy(2))
    beta = [r.dirichlet(alpha[0])]
    for i in range(1,n) :
      betai = [r.dirichlet(alpha[i])]
      beta = np.concatenate((beta,betai), axis=0)
    
    mu = [[beta[0][0]-beta[0][1], 2*(beta[0][0]+beta[0][1])-1]]
    for i in range(1,n) :
      mui = [[beta[i][0]-beta[i][1], 2*(beta[i][0]+beta[i][1])-1]]
      mu = np.concatenate((mu, mui),axis=0)
    sigma = r.wald(ksi[0], ksi[1],n)
    
    return(alpha, ksi, beta, mu, sigma)

In [301]:
def genX(n, T, mu, sigma) :
    A = np.eye(T, T+2) + mu[0][0]*np.eye(T, T+2,1) + mu[0][1]*np.eye(T, T+2,2)
    A = A@A.T
    X = [np.random.multivariate_normal(np.zeros(T), sigma[0]*A)]
    for i in range(1,n) :
      A = np.eye(T, T+2) + mu[i][0]*np.eye(T, T+2,1) + mu[i][1]*np.eye(T, T+2,2)
      A = sigma[i]*A@A.T
      Xi = [np.random.multivariate_normal(np.zeros(T), A)]
      X = np.concatenate((X,Xi), axis=0)
    return(X)

In [344]:
def MHbrouillon(N, n, T, randomstate) :
    #Génération des données observées
    alpha, ksi, beta, mu, sigma = genParam(n, randomstate)
    X = genX(n, T, mu, sigma)

    #Génération de l'état initial
    ksiMH = [np.abs(np.random.standard_cauchy(2))]
    sigmaMH = [np.random.wald(ksiMH[0], ksiMH[1], n)]

    #Génération de la chaîne de Markov
    for i in range(N) :
        L = 1

        #Génération des propositions
        ksiMHprop = [np.random.exponential(scale=ksiMH[i][0]), np.random.exponential(scale=ksiMH[i][1])]
        sigmaMHprop = []
        for k in range(n) :
            sigmaMHprop.append(np.random.exponential(scale=sigmaMH[k]))
        Lprop = 1
        
        for j in range(n):

            AMH = np.eye(T, T+2) + mu[j][0]*np.eye(T, T+2,1) + mu[j][1]*np.eye(T, T+2,2)
            AMH = sigmaMH[i][j]*(AMH@AMH.T)

            L = L*scipy.stats.multivariate_normal.pdf(X[j], mean=np.zeros(T), cov=AMH)
            L = L*scipy.stats.invgauss.pdf(sigmaMH[i][j], mu=ksiMH[i][0]/ksiMH[i][1], loc=0, scale=ksiMH[i][1])

            AMHprop = np.eye(T, T+2) + mu[j][0]*np.eye(T, T+2,1) + mu[j][1]*np.eye(T, T+2,2)
            AMHprop = sigmaMHprop[j]*(AMHprop@AMHprop.T)
            
            Lprop = Lprop*scipy.stats.multivariate_normal.pdf(X[j], mean=np.zeros(T), cov=AMHprop)
            Lprop = Lprop*scipy.stats.invgauss.pdf(sigmaMHprop[j], mu=ksiMHprop[0]/ksiMHprop[1], loc=0, scale=ksiMHprop[1])

        L = L*scipy.stats.halfcauchy.pdf(ksiMH[i][0])*scipy.stats.halfcauchy.pdf(ksiMH[i][1])
        Lprop = Lprop*scipy.stats.halfcauchy.pdf(ksiMHprop[0])*scipy.stats.halfcauchy.pdf(ksiMHprop[1])

        num = 1
        den = 1
        for k in range(n) :
            num = num*scipy.stats.expon.pdf(sigmaMH[i][k], scale=sigmaMHprop[k])
            den = den*scipy.stats.expon.pdf(sigmaMHprop[k], scale=sigmaMH[i][k])
        num = num*scipy.stats.expon.pdf(ksiMH[i][0], scale=ksiMHprop[0])*scipy.stats.expon.pdf(ksiMH[i][1], scale=ksiMHprop[1])
        den = den*(scipy.stats.expon.pdf(ksiMHprop[0], scale=ksiMH[i][0])*scipy.stats.expon.pdf(ksiMHprop[1], scale=ksiMH[i][1]))

        if den == 0 :  
            ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
            sigmaMH = np.concatenate((sigmaMH, [sigmaMH[i]]), axis=0)
        else :
            r = min(1, num/den)
            if np.random.uniform() <= r:
                ksiMH = np.concatenate((ksiMH,[ksiMHprop]), axis=0)
                sigmaMH = np.concatenate((sigmaMH, [sigmaMHprop]), axis=0)
            else : 
                ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
                sigmaMH = np.concatenate((sigmaMH, [sigmaMH[i]]), axis=0)
        return(sigmaMH[-1], ksiMH[-1])

In [353]:
def MH(N, n, T, randomstate) :
        
    #Génération des données observées
    alpha, ksi, beta, mu, sigma = genParam(n, randomstate)
    X = genX(n, T, mu, sigma)
    
    #Génération de l'état initial
    ksiMH = [np.abs(np.random.standard_cauchy(2))]
    sigmaMH = [np.random.wald(ksiMH[0][0], ksiMH[0][1], n)]
    
    #Génération de la chaîne de Markov
    for i in range(N) :
        L = 1
    
        #Génération des propositions
        ksiMHprop = [np.random.exponential(scale=ksiMH[i][0]), np.random.exponential(scale=ksiMH[i][1])]
        sigmaMHprop = []
        for k in range(n) :
            sigmaMHprop.append(np.random.exponential(scale=sigmaMH[i][k]))
        Lprop = 1
        
        for j in range(n):
    
            AMH = np.eye(T, T+2) + mu[j][0]*np.eye(T, T+2,1) + mu[j][1]*np.eye(T, T+2,2)
            AMH = sigmaMH[i][j]*(AMH@AMH.T)
    
            L = L*scipy.stats.multivariate_normal.pdf(X[j], mean=np.zeros(T), cov=AMH)
            L = L*scipy.stats.invgauss.pdf(sigmaMH[i][j], mu=ksiMH[i][0]/ksiMH[i][1], loc=0, scale=ksiMH[i][1])
    
            AMHprop = np.eye(T, T+2) + mu[j][0]*np.eye(T, T+2,1) + mu[j][1]*np.eye(T, T+2,2)
            AMHprop = sigmaMHprop[j]*(AMHprop@AMHprop.T)
                
            Lprop = Lprop*scipy.stats.multivariate_normal.pdf(X[j], mean=np.zeros(T), cov=AMHprop)
            Lprop = Lprop*scipy.stats.invgauss.pdf(sigmaMHprop[j], mu=ksiMHprop[0]/ksiMHprop[1], loc=0, scale=ksiMHprop[1])
    
        L = L*scipy.stats.halfcauchy.pdf(ksiMH[i][0])*scipy.stats.halfcauchy.pdf(ksiMH[i][1])
        Lprop = Lprop*scipy.stats.halfcauchy.pdf(ksiMHprop[0])*scipy.stats.halfcauchy.pdf(ksiMHprop[1])
    
        num = 1
        den = 1
        for k in range(n) :
            num = num*scipy.stats.expon.pdf(sigmaMH[i][k], scale=sigmaMHprop[k])
            den = den*scipy.stats.expon.pdf(sigmaMHprop[k], scale=sigmaMH[i][k])
        num = num*scipy.stats.expon.pdf(ksiMH[i][0], scale=ksiMHprop[0])*scipy.stats.expon.pdf(ksiMH[i][1], scale=ksiMHprop[1])
        den = den*(scipy.stats.expon.pdf(ksiMHprop[0], scale=ksiMH[i][0])*scipy.stats.expon.pdf(ksiMHprop[1], scale=ksiMH[i][1]))
        num = num*Lprop
        den = den*L
        if den == 0 :  
            ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
            sigmaMH = np.concatenate((sigmaMH, [sigmaMH[i]]), axis=0)
            #ksiMH = np.concatenate((ksiMH,[ksiMHprop]), axis=0)
            #sigmaMH = np.concatenate((sigmaMH, [sigmaMHprop]), axis=0)
        else :
            r = min(1, num/den)
            if np.random.uniform() <= r:
                ksiMH = np.concatenate((ksiMH,[ksiMHprop]), axis=0)
                sigmaMH = np.concatenate((sigmaMH, [sigmaMHprop]), axis=0)
            else : 
                ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
                sigmaMH = np.concatenate((sigmaMH, [sigmaMH[i]]), axis=0)

        meanksiMH = [np.mean(ksiMH[:, 0]), np.mean(ksiMH[:, 1])]
        meansigmaMH = []
        for i in range(n) :
            meansigmaMH.append(np.mean(sigmaMH[:, i]))
            
    return(sigmaMH[-1], meansigmaMH, ksiMH[-1], meanksiMH)

In [354]:
MH(300, 20, 40, 0)

(array([1.10806478, 2.45067979, 0.63251223, 1.82412954, 1.06868625,
        0.84069211, 2.5584156 , 1.66300918, 0.65837664, 1.36244052,
        0.72497089, 1.029784  , 1.04896487, 2.42311849, 1.95460084,
        2.53721038, 1.69785517, 2.40588774, 2.16365657, 0.99241266]),
 [1.108064782147084,
  2.4506797924995283,
  0.6325122260194739,
  1.8241295357562792,
  1.0686862478512162,
  0.8406921142295956,
  2.5584156028233442,
  1.663009176186943,
  0.6583766408380003,
  1.3624405232299188,
  0.7249708934834969,
  1.029783999955144,
  1.0489648724329081,
  2.4231184904127936,
  1.9546008353539421,
  2.5372103773278627,
  1.6978551718116586,
  2.4058877429349184,
  2.163656565703697,
  0.9924126583881038],
 array([1.27447194, 7.30330571]),
 [1.274471939477839, 7.3033057086871835])

In [356]:
sigma

array([0.78105041, 0.3353283 , 0.43766633, 0.99722868, 1.64719447,
       0.42325282, 1.58275635, 0.63839199, 1.04385578, 1.64553035])

In [55]:
def probmu2(x, u, v) :
  if (x<-1) | (x>1) :
    return(0)
  else :
    return(2*(((x+1)/2)**(u-1))*((1-(x+1)/2)**(v-1))/beta(u,v))

In [178]:
def Normpdf(X, mean, cov) :
    y =  np.exp(
            (-1/2)*np.dot((X[0]-mean), np.dot(np.linalg.inv(A),(X[0]-mean)))
                         )
    y = y/np.linalg.det(A)**(1/2)
    y = y/(2*np.pi)**(len(X[0])/2)
    
    return(y)

In [294]:
ksiMH = [np.abs(np.random.standard_cauchy(2))]
for i in range(0, 1000) :
    L = 1
    ksiMHprop = [np.random.exponential(scale=ksiMH[i][0]), np.random.exponential(scale=ksiMH[i][1])]
    Lprop = 1
    for j in range(len(sigma)):
        L = L*scipy.stats.invgauss.pdf(sigma[j], mu=ksiMH[i][0]/ksiMH[i][1], loc=0, scale=ksiMH[i][1])
        Lprop = Lprop*scipy.stats.invgauss.pdf(sigma[j], mu=ksiMHprop[0]/ksiMHprop[1], loc=0, scale=ksiMHprop[1])
    prior = scipy.stats.halfcauchy.pdf(ksiMH[i][0])*scipy.stats.halfcauchy.pdf(ksiMH[i][1])
    priorprop = scipy.stats.halfcauchy.pdf(ksiMHprop[0])*scipy.stats.halfcauchy.pdf(ksiMHprop[1])
    kernel = scipy.stats.expon.pdf(ksiMH[i][0], scale=ksiMHprop[0])*scipy.stats.expon.pdf(ksiMH[i][1], scale=ksiMHprop[1])
    kernel = kernel/(scipy.stats.expon.pdf(ksiMHprop[0], scale=ksiMH[i][0])*scipy.stats.expon.pdf(ksiMHprop[1], scale=ksiMH[i][1]))
    if L*prior == 0 :
        ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
    else :
        r = min(1, Lprop*priorprop*kernel/(L*prior))
        if np.random.uniform() <= r:
            ksiMH = np.concatenate((ksiMH,[ksiMHprop]), axis=0)
        else : 
            ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)


In [295]:
m0 = 0
m1 = 0
for i in range(100, ksiMH.shape[0]):
    m0+= ksiMH[i][0]
    m1+= ksiMH[i][1]
m0 = m0/(ksiMH.shape[0]-100)
m1 = m1/(ksiMH.shape[0]-100)